In [41]:
import time
import requests
import fiona
import geopandas as gpd
import pandas as pd
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from tqdm import tqdm_notebook as tqdm
import psycopg2
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import os
from tools_Scrape import scrape, explode

In [42]:
# Start the clock
start_time = time.time()

In [43]:
# QuerySettings
querysize = 100
offsetmax = 245000
offsets = [str(x) for x in range(1, offsetmax, querysize)]

In [44]:
# Set Paths and Table Names
querysize = str(querysize)
for queryoffset in tqdm(offsets):
    try:
        buildingsAPI = 'https://www.portlandmaps.com/arcgis/rest/services/Public/COP_OpenData_Property/MapServer/184/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset='\
                        + queryoffset + '&resultRecordCount='\
                        + querysize + '&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=geojson'
        api = buildingsAPI
        exportname = "buildings"
        tablename = exportname
        gdf = scrape(api,tablename)
    except:
        pass

In [45]:
# # Sample Query for Check
# with engine.connect() as conn:
#     sql = """SELECT COUNT(geom) FROM """ + tablename + """ LIMIT 1;"""
#     results = conn.execute(sql)
#     for result in results:
#         print result
#     conn.close()

In [46]:
# Export the Query for Mapbox
con = psycopg2.connect(database='tempspatial', user='zephschafer')
query = 'SELECT * FROM ' + tablename + ';'
gdf = gpd.GeoDataFrame.from_postgis(query, con, geom_col='geom' )
gdf['map_height'] = gdf['avg_height']/3.28084
use_type = pd.DataFrame({'bldg_use':['Multi Family Residential'\
                                    ,'Commercial Restaurant'\
                                    ,'Institutional'\
                                    ,'Commercial Office'\
                                    ,'Industrial','Commercial Retail'\
                                    ,'Vacant','Commercial Hotel'\
                                    ,'Parking']\
                       ,'map_color':['rgb(102,205,170)','rgb(0,0,156)'\
                                     ,'rgb(166,42,42)','rgb(0,0,156)'\
                                     ,'rgbrgb(102,205,170)'\
                                     ,'rgb(0,100,100)','rgb(102,205,170)'\
                                     ,'rgb(102,205,170))'\
                                     ,'rgb(102,205,170)']})
gdf = gdf.merge(use_type, on='bldg_use', how='inner')

In [47]:
try:
    os.remove('../data/' + exportname + '.geojson')
except:
    pass
gdf.to_file('../data/' + exportname + '.geojson', driver = 'GeoJSON')

In [48]:
stop_time = time.time()
print 'This notebook took ' \
        + str(int((stop_time-start_time)/60/60)) + ' Hours ' \
        + str(int((stop_time-start_time)/60)) + ' Minutes ' \
        + 'and ' + str(int(stop_time-start_time)%60) + ' Seconds'

This notebook took 2 Hours 168 Minutes and 51 Seconds
